<a href="https://colab.research.google.com/github/Naveen-Chaurasia/Generative-AI-POCs/blob/main/Neo4j_Langchain_POC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install langchain
!pip install neo4j
from langchain.chains import GraphCypherQAChain
from langchain.chat_models import ChatOpenAI
from langchain.graphs import Neo4jGraph

graph = Neo4jGraph(
    url="neo4j+s://ac4ba3f7.databases.neo4j.io", username="neo4j", password="***********"
)

In [5]:
# graph.refresh_schema()
print(graph.schema)

Node properties are the following:
Option {eci_cost: FLOAT, pocp: FLOAT, tetp: FLOAT, maetp: FLOAT, odp: FLOAT, gwp_100_ebc: FLOAT, htp: FLOAT, faetp: FLOAT, gwp_100: FLOAT, ap: FLOAT, ep: FLOAT, adp_elements: FLOAT, adp_fossil: FLOAT, name: STRING, uid: STRING},Scope {eci_cost: FLOAT, pocp: FLOAT, tetp: FLOAT, maetp: FLOAT, odp: FLOAT, gwp_100_ebc: FLOAT, htp: FLOAT, faetp: FLOAT, gwp_100: FLOAT, ap: FLOAT, ep: FLOAT, adp_elements: FLOAT, adp_fossil: FLOAT, uid: STRING, name: STRING},Process {tetp: FLOAT, eci_cost: FLOAT, odp: FLOAT, pocp: FLOAT, htp: FLOAT, maetp: FLOAT, gwp_100: FLOAT, gwp_100_ebc: FLOAT, ep: FLOAT, faetp: FLOAT, adp_fossil: FLOAT, ap: FLOAT, functional_unit: STRING, adp_elements: FLOAT, uid: STRING, name: STRING},Material {tetp: FLOAT, eci_cost: FLOAT, odp: FLOAT, pocp: FLOAT, htp: FLOAT, maetp: FLOAT, gwp_100: FLOAT, gwp_100_ebc: FLOAT, ep: FLOAT, faetp: FLOAT, adp_fossil: FLOAT, ap: FLOAT, functional_unit: STRING, adp_elements: FLOAT, uid: STRING, name: STRING},E

In [11]:
# !pip install huggingface_hub
repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
from langchain.llms import HuggingFaceHub
llm=HuggingFaceHub(huggingfacehub_api_token='*********',
                     repo_id=repo_id, model_kwargs={"temperature":0.2, "max_new_tokens":50})
chain = GraphCypherQAChain.from_llm(llm, graph=graph, verbose=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [22]:
chain.run("which material is accounting for minimum gwp emission?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:


MATCH (m:Material)
WITH m, sum(m.gwp_100) as total_gwp
ORDER BY total_gwp ASC
LIMIT 1
RETURN m.name as material_
Full Context:
[{'material_': 'UPR Based Gelcoat'}]

> Finished chain.


' The material accounting for minimum GWP emission is UPR Based Gelcoat.'

In [24]:
chain.run("give material which have highest eci cost")



> Entering new GraphCypherQAChain chain...
Generated Cypher:


MATCH (m:Material)
WITH m
ORDER BY m.eci_cost DESC
LIMIT 1
RETURN m
Full Context:
[{'m': {'maetp': 0.009542, 'htp': 1.4268e-07, 'adp_elements': 2.43714e-06, 'gwp_100_ebc': 20.707, 'ep': 0.000158497, 'faetp': 0.000158497, 'ap': 0.03031, 'functional_unit': 'm2', 'uid': 'Fire Liner', 'eci_cost': 1.2429910404846005, 'pocp': 0.02923, 'name': 'Fire Liner', 'odp': 8.6510400178e-09, 'gwp_100': 20.707, 'tetp': 0.10209, 'adp_fossil': 264.58}}]

> Finished chain.


' The material with the highest ECI cost is the Fire Liner, with an ECI cost of 1.24.'

In [30]:
chain.run("Which option has lowest gwp")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
_100_ebc?

The answer is:
MATCH (o:Option)-[:CONTAINS]->(s:Scope)-[:USES]->(p:Process)
WHERE NOT (p)-[:HAS


ValueError: ignored